## Testing/building functions section

In [ ]:
def get_dx_cols(df):
    '''
    This function retrieves DX (diagnosis) column names from a given dataset
    '''
    cols_all = df.columns
    rdx = re.compile(".*DX\d")
    cols_dx = list(filter(rdx.match, cols_all))
    return cols_dx

def get_pr_cols(df):
    '''
    This function retrieves PR (procedure) column names from a given dataset
    '''
    cols_all = df.columns
    rpr = re.compile(".*PR\d")
    cols_pr = list(filter(rpr.match, cols_all))
    return cols_pr

def filter_chunk_by_dx(df, dx_icds):
    '''
    This function isolates rows from a df chunk based on the input of DX ICD codes provided to the function
    '''
    extract_list = []
    cols_dx = get_dx_cols(df)
    for i in cols_dx:
        extract = df.loc[df[i].isin(dx_icds)]
        extract_list.append(extract)
    extract_concat_dedup = pd.concat(extract_list).drop_duplicates()
    return extract_concat_dedup

def filter_chunk_by_pr(df, pr_icds):
    '''
    This function isolates rows from a df chunk based on the input of PR ICD codes provided to the function
    '''
    extract_list = []
    cols_pr = get_pr_cols(df)
    for i in cols_pr:
        extract = df.loc[df[i].isin(pr_icds)]
        extract_list.append(extract)
    extract_concat_dedup = pd.concat(extract_list).drop_duplicates()
    return extract_concat_dedup

def filter_NIS_year_by_dx(nis_df, dx_icds):
    '''
    This function isolates rows from a full year df based on the input of DX ICD codes provided to the function
    '''
    dfs = [] # holds data chunks
    for chunk in nis_df:
        chunk_filtered = filter_chunk_by_dx(chunk, dx_icds)
        dfs.append(chunk_filtered)
        clear_output(wait=True)
        print(f"Filtered {len(dfs)*100000} rows...")
        if len(dfs) == 3:
            break
    print(f"Finished filtering, now concatening all chunks...")
    data = pd.concat(dfs)
    print(f"Finished concatening!")
    return data

def filter_NIS_year_by_pr(nis_df, pr_icds):
    '''
    This function isolates rows from a full year df based on the input of PR ICD codes provided to the function
    '''
    dfs = [] # holds data chunks
    for chunk in nis_df:
        chunk_filtered = filter_chunk_by_pr(chunk, pr_icds)
        dfs.append(chunk_filtered)
        clear_output(wait=True)
        print(f"Filtered {len(dfs)*100000} rows...")
#        if len(dfs) == 3:
#            break
    print(f"Finished filtering, now concatening all chunks...")
    data = pd.concat(dfs)
    print(f"Finished concatening!")
    return data

## Code running section

In [2]:
# at top of notebook to auto-relaod
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyreadstat.pyreadstat as ps
from scipy import stats
import os
#import re
#from IPython.display import clear_output
from ClinicalResearch.data_utils import get_dx_cols, get_pr_cols, filter_chunk_by_dx, filter_chunk_by_pr, filter_NIS_year_by_dx, filter_NIS_year_by_pr

In [4]:
# initiate ICD lists for MAX_POP
dx_icds = ['56211', '56213', 'K5720', 'K5721', 'K5732', 'K5732', 'K5733', 'K5740', 'K5752', 'K5780', 'K5792']
pr_icds = ["4862", "461", "4610", "4611", "4612", "4613", "4614", "0D1N0Z4", "0D1N4Z4", "0D1L0Z4", "0D1L4Z4", "0D1M0Z4", "0D1M4Z4"]

In [5]:
# Read SAS file in chunks
sasfile_nis16 = "../raw_data/nis_2016_combined_std.sas7bdat"
nis16 = pd.read_sas(sasfile_nis16, chunksize=100000, iterator=True, format = 'sas7bdat', encoding="latin-1")

In [6]:
cohort16 = filter_NIS_year_by_dx(nis16, dx_icds)

Filtered 300000 rows...
Finished filtering, now concatening all chunks...
Finished concatening!


In [7]:
cohort16

,HOSP_NIS,KEY_NIS,AGE,AGE_NEONATE,AMONTH,AWEEKEND,DIED,DISCWT,DISPUNIFORM,DQTR,...,APRDRG_Severity,HOSP_BEDSIZE,HOSP_LOCTEACH,HOSP_REGION,H_CONTRL,N_DISC_U,N_HOSP_U,S_DISC_U,S_HOSP_U,TOTAL_DISC
5,10001.0,10002081.0,87.0,NaN,6.0,1.0,0.0,4.999956,1.0,2.0,...,2.0,1.0,2.0,1.0,2.0,113179.0,29.0,22636.0,25.0,1489.0
17,10001.0,10005354.0,54.0,NaN,3.0,0.0,0.0,4.999956,1.0,1.0,...,1.0,1.0,2.0,1.0,2.0,113179.0,29.0,22636.0,25.0,1489.0
42,10001.0,10011012.0,41.0,NaN,8.0,0.0,0.0,4.999956,1.0,3.0,...,1.0,1.0,2.0,1.0,2.0,113179.0,29.0,22636.0,25.0,1489.0
72,10001.0,10017663.0,30.0,NaN,2.0,0.0,0.0,4.999956,1.0,1.0,...,1.0,1.0,2.0,1.0,2.0,113179.0,29.0,22636.0,25.0,1489.0
260,10001.0,10060931.0,54.0,NaN,4.0,1.0,0.0,4.999956,1.0,2.0,...,2.0,1.0,2.0,1.0,2.0,113179.0,29.0,22636.0,25.0,1489.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224770,10122.0,10089984.0,62.0,NaN,9.0,0.0,1.0,5.000010,20.0,3.0,...,4.0,3.0,3.0,1.0,2.0,493381.0,12.0,98676.0,11.0,6536.0
229227,10122.0,10312735.0,78.0,NaN,2.0,0.0,0.0,5.000010,5.0,1.0,...,4.0,3.0,3.0,1.0,2.0,493381.0,12.0,98676.0,11.0,6536.0
242413,10133.0,10284624.0,63.0,NaN,6.0,0.0,0.0,4.999872,5.0,3.0,...,4.0,2.0,2.0,1.0,2.0,78003.0,11.0,15601.0,9.0,2406.0
284069,10157.0,10250231.0,62.0,NaN,9.0,0.0,0.0,5.000030,1.0,4.0,...,3.0,1.0,3.0,1.0,2.0,329532.0,41.0,65906.0,39.0,1084.0


In [13]:
print(cohort16.columns)

Index(['HOSP_NIS', 'KEY_NIS', 'AGE', 'AGE_NEONATE', 'AMONTH', 'AWEEKEND',
       'DIED', 'DISCWT', 'DISPUNIFORM', 'DQTR',
       ...
       'APRDRG_Severity', 'HOSP_BEDSIZE', 'HOSP_LOCTEACH', 'HOSP_REGION',
       'H_CONTRL', 'N_DISC_U', 'N_HOSP_U', 'S_DISC_U', 'S_HOSP_U',
       'TOTAL_DISC'],
      dtype='object', length=110)


In [11]:
get_dx_cols(cohort16)

['DX1',
 'DX2',
 'DX3',
 'DX4',
 'DX5',
 'DX6',
 'DX7',
 'DX8',
 'DX9',
 'DX10',
 'DX11',
 'DX12',
 'DX13',
 'DX14',
 'DX15',
 'I10_DX16',
 'I10_DX17',
 'I10_DX18',
 'I10_DX19',
 'I10_DX20',
 'I10_DX21',
 'I10_DX22',
 'I10_DX23',
 'I10_DX24',
 'I10_DX25',
 'I10_DX26',
 'I10_DX27',
 'I10_DX28',
 'I10_DX29',
 'I10_DX30']

In [ ]:
d1.to_csv("sas_columns.csv", header=True, index=False)
d2.to_csv("csv_columns.csv", header=True, index=False)

In [ ]:
d3 = ps.read_sas7bdat(file2,row_limit=1000000)[0]
d3

In [ ]:
d3.shape

https://github.com/Roche/pyreadstat#reading-rows-in-chunks

In [ ]:
d4 = ps.read_file_in_chunks(ps.read_sas7bdat, file2, chunksize=1000, limit=10000)

In [ ]:
d4[0]